<a href="https://colab.research.google.com/github/bdipesh3045/356-Days-Of-Data-Science/blob/main/Untitled9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("hw.csv", on_bad_lines='skip', engine='python')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Comment    144 non-null    object 
 1   Permalink  0 non-null      float64
 2   Links      0 non-null      float64
dtypes: float64(2), object(1)
memory usage: 3.5+ KB


In [ ]:
df['Comment'][0]

"School: Small R-2 State School Major/Degree Program: ChemE Expected Graduation Date: Spring '28 GPA: 3.5 Relevant Coursework: Calc 1 & 2, Gen Chem 1 & 2, Physics 1 & 2 (mechanics and electricity) Demographic: Cis Black Male Research Experience: none Past REUs: none Applied: MS State USDA-NIFA REEU (accepted)"

In [ ]:
!pip install -q -U google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.5/260.5 kB 19.8 MB/s eta 0:00:00


In [ ]:

from google import genai
from pydantic import BaseModel, Field
from typing import List, Optional
import json

client = genai.Client(api_key="")



In [ ]:





class Selected(BaseModel):
    select: str = Field(description="Name of the research program selected.")



class Applied(BaseModel):
    applied: str = Field(description="Name of research program appied into.")

class Summar(BaseModel):
    Degree: str = Field(description="Current program you applied into")
    applied_data: List[Selected]
    selected_data: List[Applied]

class Summary(BaseModel):
  data:str= List[Summar]


In [ ]:
def prompt_data(df: pd.DataFrame):
    csv_data = df.to_csv(index=False)
    contents=f"""
            Convert the CSV into JSON that matches this exact schema:
            Strict dont skip any output No yapping
            Degree: string
            Program List: string
            Applied: string[]
            Selected: string[]

            Return ONLY valid JSON.

            CSV Data:
            {csv_data}
        """
    return contents


In [ ]:
def summarize(df: pd.DataFrame):
  prompt=prompt_data(df=df)

  response = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=prompt,
  config={
  "response_mime_type": "application/json",
  "response_json_schema": Summary.model_json_schema(),
  },
  )



  return response

In [ ]:
data=summarize(df)

/usr/local/lib/python3.12/dist-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value typing.List[__main__.Summar] is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [ ]:
data

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      avg_logprobs=-0.00808987049871582,
      content=Content(
        parts=[
          Part(
            text="""{
  "data": "[{\"Degree\": \"ChemE\", \"Program List\": \"MS State USDA-NIFA REEU\", \"Applied\": [\"MS State USDA-NIFA REEU\"], \"Selected\": [\"MS State USDA-NIFA REEU\"]}, {\"Degree\": \"Computer Science\", \"Program List\": \"UMBC NSF-REU - Smart Computing and Communications, UMBC iHARP REU, HHMI SURP\", \"Applied\": [\"UMBC NSF-REU - Smart Computing and Communications\", \"UMBC iHARP REU\", \"HHMI SURP\"], \"Selected\": [\"UMBC NSF-REU - Smart Computing and Communications\"]}, {\"Degree\": \"Computer Science and Mathematics\", \"Program List\": \"Rutgers DIMACS\", \"Applied\": [\"Rutgers DIMACS\"], \"Selected\": [\"Rutgers DIMACS\"]}, {\"Degree\": \"Biochemistry\", \"Program List\": \"Indiana University School of Medicine SUREBS, University of Illinois Chicago SURF, Northw

In [ ]:
json_output = data.model_dump_json(by_alias=True, indent=2)

In [ ]:
import json

data_dict = json.loads(json_output)
data
print(data_dict.keys())
data=data_dict.get('candidates')


dict_keys(['sdkHttpResponse', 'candidates', 'createTime', 'modelVersion', 'promptFeedback', 'responseId', 'usageMetadata', 'automaticFunctionCallingHistory', 'parsed'])


In [ ]:
for i in data[0]:
  print(i)

content
citationMetadata
finishMessage
tokenCount
finishReason
avgLogprobs
groundingMetadata
index
logprobsResult
safetyRatings
urlContextMetadata


In [ ]:
for i in data[0]["content"]["parts"]:
  final=(i.get("text"))
final

'{\n  "data": "[{\\"Degree\\": \\"ChemE\\", \\"Program List\\": \\"MS State USDA-NIFA REEU\\", \\"Applied\\": [\\"MS State USDA-NIFA REEU\\"], \\"Selected\\": [\\"MS State USDA-NIFA REEU\\"]}, {\\"Degree\\": \\"Computer Science\\", \\"Program List\\": \\"UMBC NSF-REU - Smart Computing and Communications, UMBC iHARP REU, HHMI SURP\\", \\"Applied\\": [\\"UMBC NSF-REU - Smart Computing and Communications\\", \\"UMBC iHARP REU\\", \\"HHMI SURP\\"], \\"Selected\\": [\\"UMBC NSF-REU - Smart Computing and Communications\\"]}, {\\"Degree\\": \\"Computer Science and Mathematics\\", \\"Program List\\": \\"Rutgers DIMACS\\", \\"Applied\\": [\\"Rutgers DIMACS\\"], \\"Selected\\": [\\"Rutgers DIMACS\\"]}, {\\"Degree\\": \\"Biochemistry\\", \\"Program List\\": \\"Indiana University School of Medicine SUREBS, University of Illinois Chicago SURF, Northwestern SROP, Rush Summer Scholars Program, Northwestern SynBreu, UChicago MGCB, Van Andel Insitute Summer Research Program\\", \\"Applied\\": [\\"India

In [ ]:
final=final[14:]
prompt=f"Act as excel file repair expert and your main goal is to organize the the given csv data Also make sure all the brackets are closed properly The csv data is given below{final}  no yapping"


from typing import List
from typing_extensions import TypedDict, Literal


class ProgramEntry(TypedDict):
    major: str
    degree_level: Literal["undergraduate", "graduate"]
    programs_applied: List[str]
    programs_selected: List[str]

class ProgramData(TypedDict):
  data: List[ProgramEntry]



In [ ]:

response = client.models.generate_content(
model="gemini-2.5-pro",
contents=prompt,
config={
"response_mime_type": "application/json",
          "response_schema": ProgramData
},
  )



In [ ]:
response

GenerateContentResponse(
  automatic_function_calling_history=[],
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            text='{"data":[{"major":"Computer Science","degree_level":"undergraduate","programs_applied":["UMBC NSF-REU - Smart Computing and Communications","UMBC iHARP REU","HHMI SURP"],"programs_selected":["UMBC NSF-REU - Smart Computing and Communications"]},{"major":"Computer Science and Mathematics","degree_level":"undergraduate","programs_applied":["Rutgers DIMACS"],"programs_selected":["Rutgers DIMACS"]},{"major":"Biochemistry","degree_level":"undergraduate","programs_applied":["Indiana University School of Medicine SUREBS","University of Illinois Chicago SURF","Northwestern SROP","Rush Summer Scholars Program","Northwestern SynBreu","UChicago MGCB","Van Andel Insitute Summer Research Program"],"programs_selected":["Indiana University School of Medicine SUREBS","Van Andel Insitute Summer Research Program"]},{"major":"CS, Math Min

In [ ]:
response_json = json.loads(response.text)


try:
    # Gemini may return `response.text` or `response.parts[0].text`
    raw_json = response.text if hasattr(response, 'text') else response.parts[0].text
    data = json.loads(raw_json)
except Exception as e:
    print("Error parsing JSON:", e)


In [ ]:
data

{'data': [{'major': 'Computer Science',
   'degree_level': 'undergraduate',
   'programs_applied': ['UMBC NSF-REU - Smart Computing and Communications',
    'UMBC iHARP REU',
    'HHMI SURP'],
   'programs_selected': ['UMBC NSF-REU - Smart Computing and Communications']},
  {'major': 'Computer Science and Mathematics',
   'degree_level': 'undergraduate',
   'programs_applied': ['Rutgers DIMACS'],
   'programs_selected': ['Rutgers DIMACS']},
  {'major': 'Biochemistry',
   'degree_level': 'undergraduate',
   'programs_applied': ['Indiana University School of Medicine SUREBS',
    'University of Illinois Chicago SURF',
    'Northwestern SROP',
    'Rush Summer Scholars Program',
    'Northwestern SynBreu',
    'UChicago MGCB',
    'Van Andel Insitute Summer Research Program'],
   'programs_selected': ['Indiana University School of Medicine SUREBS',
    'Van Andel Insitute Summer Research Program']},
  {'major': 'CS, Math Minor',
   'degree_level': 'undergraduate',
   'programs_applied': [

In [ ]:
df=pd.DataFrame(data["data"])

,major,degree_level,programs_applied,programs_selected
0,Computer Science,undergraduate,[UMBC NSF-REU - Smart Computing and Communicat...,[UMBC NSF-REU - Smart Computing and Communicat...
1,Computer Science and Mathematics,undergraduate,[Rutgers DIMACS],[Rutgers DIMACS]
2,Biochemistry,undergraduate,"[Indiana University School of Medicine SUREBS,...","[Indiana University School of Medicine SUREBS,..."
3,"CS, Math Minor",undergraduate,"[UMich, Yale, UW Bothel, Baruch, Minnesota, Wo...","[Polymath Jr., Georgia State GDA]"
4,CS/DS,undergraduate,"[UCR, Colorado Springs, UW Eau-Claire, UT iCom...","[UCR, Colorado Springs, Eau-Claire, WashU, ND ..."
5,General AAOT & BioResource Research,undergraduate,"[The Morton Arboretum CROPPS, Harvard GEO, UCR...",[Harvard GEO]
6,Computer Science and Political Science,undergraduate,"[Georgetown CS/Public Policy REU, CMU REUSE]",[Georgetown CS/Public Policy REU]
7,"Computer science, Math",undergraduate,[Portland State Computational Modeling Serving...,[Portland State Computational Modeling Serving...
8,Geology and Biology,undergraduate,"[Harvard GEO, U of Oklahoma: Balkans/Permian p...",[USM Gulf Coastal Plain Ecology]
9,ChemE,undergraduate,[Interdisciplinary Renewable and Environmental...,[UND]


In [ ]:
# Optional: convert lists to comma-separated text for nicer Excel formatting
df['programs_applied'] = df['programs_applied'].apply(lambda x: ", ".join(x))
df['programs_selected'] = df['programs_selected'].apply(lambda x: ", ".join(x))

In [ ]:

# Save to Excel
df.to_excel("programs_output.xlsx", index=False)

print("Excel file saved as programs_output.xlsx")

Excel file saved as programs_output.xlsx
